In [1]:
#-----------------------------------------
#       German Hernández-2171842
#-----------------------------------------
import numpy as np
from matplotlib import pyplot as plt
import sympy as sp
from sympy.parsing.sympy_parser import parse_expr

In [2]:
#Programa para resolver sistema 
#de ecuaciones multivaribale por metodo de
#N-R

def NR_multi(V_i,X,v_F,error):
	'''V_i: Vector con valores iniciales.
	   X: Tupla con simbolos a utilizar, cada simbolo 
		  se debe definir entre comillas.
	   v_F: Vector que contiene funciones que definen 
			el sistema de ecuaciones no lineales a resolver,
			cada funcion debe definirse entre comillas.'''
	#Se procede a convertir simbolos str a simbolos de SymPy
	v_Var=sp.symbols(X)
	#Se convierten funciones de str a funciones operables por SymPy
	#Se utiliza una lista comprimida
	Func=[parse_expr(v_F[k]) for k in list(range(0,len(v_F)))] 
	#Se define el Jacobiano
	m_J=[]# Matriz donde se guardaran derivadas
	#Inicia ciclo for para crear Jacobiano y convertir 
	#lista de funciones a columna
	c_F=[]
	for k in list(range(0,len(v_F))):
		v_Deriv=[]
		funcion=Func[k]
		#Ciclo for anidado para ingresar derivadas por filas
		for m in list(range(0,len(v_Var))):
			d_F=sp.diff(funcion,v_Var[m])
			v_Deriv.append(d_F)
		#Fin de for anidado
		c_F.append([Func[k]])
		m_J.append(v_Deriv)
	#Fin de ciclo for
	#Se convierten funciones a expresiones evaluables, que reciben
	#la lista de valores iniciales
	Func_eval=sp.lambdify([v_Var],sp.Matrix(c_F))
	mJ_eval=sp.lambdify([v_Var],sp.Matrix(m_J))
	#Se definen condiciones de iteracion
	V_ic=np.asarray(V_i).reshape(-1,1)#Se transforma vector a matriz
	#con una columna
	Xi=V_ic-np.dot(np.linalg.inv(mJ_eval(V_i)),Func_eval(V_i))
	sum_e=np.sum(np.abs(Xi-V_ic))
	#Inicia ciclo while para solucionar problema
	n=0
	while (sum_e>error):
		n=n+1
		V_i=Xi
		V_if=list(V_i.reshape(1,-1)[0])
		Xi=V_i-np.dot(np.linalg.inv(mJ_eval(V_if)),Func_eval(V_if))
		sum_e=np.sqrt(np.sum((Xi-V_i)**2))
	#Fin de while
	return Xi,sum_e,n
print(NR_multi([0.5,0.1],("XR","NR"),["-23.591*XR**5 + 42.497*XR**4 - 26.771*XR**3 + 6.3438*XR**2 - 1.0597*XR + 2.5007-NR","-(77/977)+(3850/977)*XR-NR"],0.00001))


(array([[0.54911902],
       [2.08506473]]), 1.874730893226378e-09, 2)


[0.5500030718731292]

In [12]:

 #ES NECESARIO UTILIZAR OTRO TIPO DE SOLVER EN ESTE CASO SE USO NRMV
#----Funciones para calculos de varias etapas en corriente directa con retencion variable y NE=0----------
#Se debe modificar la funcion de retencion en cada una de las funciones.
def Etapa1(R0,xS,xI,Yen1,Nen1,Epn_1):
	R0p=R0*(1-xI)
	xR0=(R0*xS/R0p)
	N_R0=(R0*xI/R0p)
	#Resolver sistema de ec [1]  #SE DEBE RESOLVER ESTE SISTEMA A MANO E INGRESAR a1 y b1 en NRMV dado que recibe str
	#a,b=sp.symbols("a,b")
	#e1=sp.Eq(a+b*xR0,N_R0)
	#e2=sp.Eq(a+b*Nen1,Yen1)
	#sol=sp.solve([e1,e2],a,b) #arroja un diccionario con la solucion
	#a1=sol[a]
	#b1=sol[b]
	#Ec bce de masa
	#Bce=a1+b1*X
	#Resolver sistema de ecuaciones para hallar C_pto minimo de mezcla
	Xm,Nm=sp.symbols("Xm,Nm")
	sol_2=NR_multi([1,1],("Xm","Nm"),["0.825*Xm**3-1.255*Xm**2-0.078*Xm+2-Nm","0+(3/2)*Xm-Nm"],0.00001)
	Xmin=sol_2[0][0]
	Nmin=sol_2[0][1]

	#Para las 3 etapas con E'n+1=400
	Epn1m=R0p*((xR0-Xmin)/(Xmin-Yen1))
	alpha=Epn_1/Epn1m
	E_n1P=Epn1m*alpha
	if alpha>1:
		#1
		X_M=(R0p*xR0+E_n1P*Yen1)/(R0p+E_n1P)
		N_M=(R0p*N_R0+E_n1P*Nen1)/(R0p+E_n1P)
		#2
		X_R1=X_M
		#3 Concepto de etapa ideal
		Y_E1=X_R1
		#Funcion de retencion en XR1
		N_R1=0.825*X_R1**3-1.255*X_R1**2-0.078*X_R1+2
		#Función de arrastre
		N_E=0

		#6 Resolver sistema
		R1_P=(R0p*N_R0+E_n1P*Nen1)/(N_R1)
		E1_P=(R0p*xR0-R1_P*X_R1)/(Y_E1)
		#7 Cerrar el balance
		B=(R0p+E_n1P)-(R1_P+E1_P)
		print("Cierra el balance:",abs(B))
	else:		
		print("Error")
	#Composiciones y corrientes totales:
	R1=R1_P*(1+N_R1)
	E1=E1_P*(1+N_E)
	#Composiciones pesado
	Xsr1=X_R1/(1+N_R1)
	Xdr1=(1-X_R1)/(1+N_R1)
	Xir1=(N_R1)/(1+N_R1)
	#Composiciones ligero
	Yse1=Y_E1/(1+N_E)
	Yde1=(1-Y_E1)/(1+N_E)
	Yie1=N_E/(1+N_E)
	return R1_P,E1_P,R1,E1,Xsr1,Xdr1,Xir1,Yse1,Yde1,Yie1

#Funcion para calculos de la segunda etapa en corriente directa con r=k y NE=0
#Se debe modificar si es necesario 
def Etapa2(R0,xS,xI,Yen1,Nen1,Epn_1):
	Datos=Etapa1(R0,xS,xI,Yen1,Nen1,Epn_1)
	R1p=Datos[0]
	E_n1P2=Epn_1
	xR1=Datos[4]/(Datos[4]+Datos[5])
	N_R1=Datos[6]/(Datos[4]+Datos[5])
	X_M2=(Datos[0]*xR1+Epn_1*Yen1)/(Datos[0]+Epn_1)	
	N_M2=(Datos[0]*N_R1+Epn_1*Nen1)/(Datos[0]+Epn_1)	

	X_R2=X_M2
	#3 Concepto de etapa ideal
	Y_E2=X_R2
	#Funcion de retencion en XR1
	N_R2=0.825*X_R2**3-1.255*X_R2**2-0.078*X_R2+2

	#Función de arrastre
	N_E=0

	#6 Resolver sistema
	R2_P=(R1p*N_R1+E_n1P2*Nen1)/(N_R2)
	E2_P=(R1p*xR1-R2_P*X_R2)/(Y_E2)
	#7 Cerrar el balance
	B=(R1p+E_n1P2)-(R2_P+E2_P)
	print("Cierra el balance:",abs(B))
	#Composiciones y corrientes totales:
	R2=R2_P*(1+N_R2)
	E2=E2_P*(1+N_E)
	#Composiciones pesado
	Xsr2=X_R2/(1+N_R2)
	Xdr2=(1-X_R2)/(1+N_R2)
	Xir2=(N_R2)/(1+N_R2)
	#Composiciones ligero
	Yse2=Y_E2/(1+N_E)
	Yde2=(1-Y_E2)/(1+N_E)
	Yie2=N_E/(1+N_E)	
	return R2_P,E2_P,R2,E2,Xsr2,Xdr2,Xir2,Yse2,Yde2,Yie2 #R1p,E_n1P2,xR1,N_R1,X_M2


#Esta funcion es para hacer los calculos necesarios en la ultima etapa 
#se debe modificar en Datos=funcion_anterior  en caso de que sean mas etapas
def Etapaf(R0,xS,xI,Yen1,Nen1,Epn_1):
	Datos=Etapa2(R0,xS,xI,Yen1,Nen1,Epn_1)
	R1p=Datos[0]
	E_n1P2=Epn_1
	xR1=Datos[4]/(Datos[4]+Datos[5])
	N_R1=Datos[6]/(Datos[4]+Datos[5])
	X_M2=(Datos[0]*xR1+Epn_1*Yen1)/(Datos[0]+Epn_1)	
	N_M2=(Datos[0]*N_R1+Epn_1*Nen1)/(Datos[0]+Epn_1)	


	X_R2=X_M2
	#3 Concepto de etapa ideal
	Y_E2=X_R2
	#Funcion de retencion en XR1
	N_R2=0.825*X_R2**3-1.255*X_R2**2-0.078*X_R2+2
	#Función de arrastre
	N_E=0

	#6 Resolver sistema
	R2_P=(R1p*N_R1+E_n1P2*Nen1)/(N_R2)
	E2_P=(R1p*xR1-R2_P*X_R2)/(Y_E2)
	#7 Cerrar el balance
	B=(R1p+E_n1P2)-(R2_P+E2_P)
	print("Cierra el balance:",abs(B))
	#Composiciones y corrientes totales:
	R2=R2_P*(1+N_R2)
	E2=E2_P*(1+N_E)
	#Composiciones pesado
	Xsr2=X_R2/(1+N_R2)
	Xdr2=(1-X_R2)/(1+N_R2)
	Xir2=(N_R2)/(1+N_R2)
	#Composiciones ligero
	Yse2=Y_E2/(1+N_E)
	Yde2=(1-Y_E2)/(1+N_E)
	Yie2=N_E/(1+N_E)	
	return R2_P,E2_P,R2,E2,Xsr2,Xdr2,Xir2,Yse2,Yde2,Yie2

print(Etapaf(1000,0.4,0.6,0,0,750))	

Cierra el balance: [0.]
Cierra el balance: [0.]
Cierra el balance: [0.]
(array([300.52167448]), array([752.66193415]), array([900.52167448]), array([752.66193415]), array([0.01006579]), array([0.32365375]), array([0.66628046]), array([0.03016242]), array([0.96983758]), 0.0)
